In [ ]:
#!pip install pandas_datareader

In [ ]:
#!pip install yfinance

In [ ]:
#!pip install html5lib

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from unidecode import unidecode
import re
import os
import yfinance as yf
from time import sleep
import glob

## Selenium

### Configurando WebDriver

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

### Configurando pasta de download

In [3]:
chromeOptions = webdriver.ChromeOptions()
download_path = r'C:\Users\Pedro\OneDrive\Desktop\Ironhack\04. GitHub\data_visualization\data'
prefs = {"download.default_directory" : download_path}
chromeOptions.add_experimental_option("prefs",prefs)

In [4]:
driver = webdriver.Chrome(options=chromeOptions)

## Index's list

IBOV: Índice Bovespa - É o principal indicador de desempenho das ações negociadas na B3 e reúne as empresas mais importantes do mercado de capitais brasileiro

IBXX: Índice IBrX 100 - 100 ativos de maior negociabilidade e representatividade do mercado de ações brasileiro.

IBXL: Índice IBrX 50 - 50 ativos de maior negociabilidade e representatividade do mercado de ações brasileiro.

IBRA: Índice Brasil Amplo - O objetivo do IBrA é ser o indicador do desempenho médio das cotações de todos os ativos negociados no mercado a vista (lote-padrão) da B3 que atendam a critérios mínimos de liquidez e presença em pregão, de forma a oferecer uma visão ampla do mercado acionário.

IFNC: Índice Financeiro - ativos de maior negociabilidade e representatividade dos setores de intermediários financeiros, serviços financeiros diversos, previdência e seguros.

ICON: Índice de Consumo - ativos de maior negociabilidade e representatividade dos setores de consumo cíclico, consumo não cíclico e saúde.

IEEX: Índice de Energia Elétrica - ativos de maior negociabilidade e representatividade do setor de energia elétrica.

IFIX: Índice de Fundos Imobiliários - Fundos imobiliários negociados nos mercados de bolsa e de balcão organizado da B3.

IFIL: Índice de Fundos Imobiliários de Alta Liquidez - Fundos imobiliários mais líquidos negociados nos mercados de bolsa e de balcão organizado da B3.

IMAT: Índice de Materiais Básicos - ativos de maior negociabilidade e representatividade do setor de materiais básicos.

IDIV: Índice Dividendos - ativos que se destacaram em termos de remuneração dos investidores, sob a forma de dividendos e juros sobre o capital próprio.

INDX: Índice do Setor Industrial - ativos de maior negociabilidade e representatividade dos setores da atividade industrial compreendidos por materiais básicos, bens industriais, consumo cíclico, consumo não cíclico, tecnologia da informação e saúde.

IMOB: Índice Imobiliário - ativos de maior negociabilidade e representatividade dos setores da atividade imobiliária compreendidos por exploração de imóveis e construção civil.

MLCX: Índice MidLarge Cap - ativos de uma carteira composta pelas empresas de maior capitalização.

SMLL: Índice Small Cap - empresas de menor capitalização.

UTIL: Índice Utilidade Pública - ativos de maior negociabilidade e representatividade do setor de utilidade pública (energia elétrica, água e saneamento e gás).

IVBX: Índice Valor empresas bem conceituadas pelos investidores.

## Download Index file (.csv)

In [5]:
def get_index_stocks(index, wait=6):
    '''
        Recieves the Index name, download a file that contains the index tickers, and return the name of the downloaded file
    '''

    #Chrome WebDriver opens the index webside
    url = f'https://sistemaswebb3-listados.b3.com.br/indexPage/day/{index.upper()}?language=pt-br'
    driver.get(url)
    driver.implicitly_wait(wait)

    driver.find_element(By.ID, 'segment').send_keys("Setor de Atuação")
    driver.implicitly_wait(wait)
    driver.find_element(By.LINK_TEXT, 'Download').click()
    driver.implicitly_wait(wait)

    #set the directory
    os.chdir(r'C:\Users\Pedro\OneDrive\Desktop\Ironhack\04. GitHub\data_visualization\data')
    sleep(wait)
    #get the files from the selectec directory and sort them by modification date
    files = files = list(glob.glob('*csv'))
    files.sort(key=lambda x: os.path.getmtime(x), reverse=True)

    return files[0]

## Get the Index DataFrame

In [6]:
def create_df(file):
    '''
        This function receives the name of the Stock Exchange index and returns a DataFrame with all companies and their respective sectors
    '''

    #Creating DataFrame
    DataFrame = pd.read_csv(file, 
                                encoding='ISO-8859-1',
                                header=1,                   #Uses line 1 as header
                                sep=';',                    
                                decimal=',',
                                thousands='.',
                                skipfooter=2,               #Removes last 2 lines
                                engine='python',
                                index_col=False)            #Does not make first column as index
    
    #Normalizing columns
    DataFrame.columns = [re.sub('[\.()]', '', re.sub(' ', '_', unidecode(columns.lower()))) for columns in DataFrame.columns]

    return DataFrame

## Creating DataFrames

In [12]:
col_names = [
    'sector',
    'ticker',
    'name',
    'type',
    'amount',
    'percentage',
    'percentage_acum'
]

### Ibovespa

In [7]:
ibov = create_df(get_index_stocks('ibov'))
len(ibov)

92

In [14]:
ibov.columns = col_names

,sector,ticker,name,type,amount,percentage,percentage_acum
0,Bens Indls / Máqs e Equips,WEGE3,WEG,ON NM,1481593024,2.840,2.840
1,Bens Indls / Mat Transporte,EMBR3,EMBRAER,ON NM,734632205,0.509,0.509
2,Bens Indls/Transporte,AZUL4,AZUL,PN N2,327646296,0.209,2.119
3,Bens Indls/Transporte,CCRO3,CCR SA,ON NM,1115681435,0.617,2.119
4,Bens Indls/Transporte,ECOR3,ECORODOVIAS,ON NM,333657615,0.073,2.119
...,...,...,...,...,...,...,...
87,Utilidade Públ / Energ Elétrica,ENGI11,ENERGISA,UNT N2,248477689,0.540,9.471
88,Utilidade Públ / Energ Elétrica,ENEV3,ENEVA,ON NM,1561135009,0.972,9.471
89,Utilidade Públ / Energ Elétrica,EGIE3,ENGIE BRASIL,ON NM,255208461,0.494,9.471
90,Utilidade Públ / Energ Elétrica,EQTL3,EQUATORIAL,ON NM,1100513485,1.478,9.471


In [22]:
path = r'C:\Users\Pedro\OneDrive\Desktop\Ironhack\04. GitHub\data_visualization\data/'

ibov.to_csv(f'{path}IBOV.csv',
                    encoding='ISO-8859-1',
                    sep=';',
                    decimal='.',
                    index=False)

### IBrX100

In [8]:
ibrx = create_df(get_index_stocks('ibxx'))
len(ibrx)

100

In [15]:
ibrx.columns = col_names

In [21]:
path = r'C:\Users\Pedro\OneDrive\Desktop\Ironhack\04. GitHub\data_visualization\data/'

ibrx.to_csv(f'{path}IBRX100.csv',
                    encoding='ISO-8859-1',
                    sep=';',
                    decimal='.',
                    index=False)

### IBrX50

In [9]:
ibrx50 = create_df(get_index_stocks('ibxl'))
len(ibrx50)

50

In [16]:
ibrx50.columns = col_names

In [23]:
path = r'C:\Users\Pedro\OneDrive\Desktop\Ironhack\04. GitHub\data_visualization\data/'

ibrx50.to_csv(f'{path}IBRX50.csv',
                    encoding='ISO-8859-1',
                    sep=';',
                    decimal='.',
                    index=False)

### IBrA

In [10]:
ibra = create_df(get_index_stocks('ibra'))
len(ibra)

201

In [17]:
ibra.columns = col_names

In [20]:
path = r'C:\Users\Pedro\OneDrive\Desktop\Ironhack\04. GitHub\data_visualization\data/'

ibra.to_csv(f'{path}IBRA.csv',
                    encoding='ISO-8859-1',
                    sep=';',
                    decimal='.',
                    index=False)

## Verificando empresas que divergem nos dois índices - Obsoleto

In [ ]:
emp_ibrx = ibrx['codigo']
set_ibrx = set(emp_ibrx)
len(set_ibrx)

In [ ]:
emp_ibov = ibov['codigo']
set_ibov = set(emp_ibov)
len(set_ibov)

In [ ]:
emp_ibra = ibra['codigo']
set_ibra = set(emp_ibra)
len(set_ibra)

In [ ]:
print(len(set_ibra.difference(set_ibrx)))
print(set_ibra.difference(set_ibrx))

In [ ]:
print(len(set_ibov.difference(set_ibrx)))
print(set_ibov.difference(set_ibrx))

In [ ]:
print(len(set_ibrx.difference(set_ibov)))
print(set_ibrx.difference(set_ibov))

### Verificando qual empresa da lista completa não exista em 2021

In [ ]:
set_codigo = set(lista_codigo)
set_empresas_2021 = set(empresas_2021)

print(len(set_codigo.difference(set_empresas_2021)))
print(set_codigo.difference(set_empresas_2021))

## Criando DataFrame unindo os dois índices - Obsoleto

In [ ]:
df_empresas = pd.concat([ibrx, ibov]).drop_duplicates(subset='codigo', ignore_index=True).drop(['qtde_teorica',r'part_%',r'part_%acum'], axis=1)

## Criando lista com o código dos pricipais ativos

In [ ]:
## Utilizar lista do IBRA
codigo = list(set_ibra)

## Função para baixar séries históricas do site da B3 - NÃO FUNCIONA

In [ ]:
'''
url = 'https://www.b3.com.br/pt_br/market-data-e-indices/servicos-de-dados/market-data/historico/mercado-a-vista/series-historicas/'
response = requests.get(url)
html = response.content
soup = BeautifulSoup(html)
soup.find_all('select')'''

In [ ]:
driver = webdriver.Chrome(options=chromeOptions)

In [ ]:
url = f'https://www.b3.com.br/pt_br/market-data-e-indices/servicos-de-dados/market-data/historico/mercado-a-vista/series-historicas/'
driver.get(url)
driver.implicitly_wait(5)

In [ ]:
driver.find_element(By.XPATH, '//*[@id="conteudo-principal"]/div[2]/div/div/ul/li[4]/a').click()

In [ ]:
driver.find_element(By.LINK_TEXT, 'Download').click()
driver.implicitly_wait(wait)

In [ ]:
os.chdir(r'C:\Users\Pedro\OneDrive\Desktop\Ironhack\04. GitHub\data_visualization\data')

In [ ]:
files = files = list(glob.glob('*txt'))

In [ ]:
files.sort(key=lambda x: os.path.getmtime(x), reverse=True)

## Unzip files

In [ ]:
import zipfile
path = r'C:\Users\Pedro\OneDrive\Desktop\Ironhack\04. GitHub\data_visualization\data/'

i = 1986

while i < 2000:
    with zipfile.ZipFile(f'{path}COTAHIST_A{i}.zip', 'r') as zip_ref:
        zip_ref.extractall(path)
    i +=1



## Rename files

In [ ]:
os.listdir()

In [ ]:
i = 2000

while i < 2022:
    os.rename(f'COTAHIST_A{i}.TXT', f'COTAHIST_A{i}.txt')
    i +=1

## Setting configurations to read B3 historical files

In [ ]:
pd.set_option('display.max_columns', 500)
pd.options.display.max_columns=500

In [ ]:
path = r'C:\Users\Pedro\OneDrive\Desktop\Ironhack\05. Dados\B3\txt/'

year = 2022

widths = [2,8,2,12,3,12,10,3,4,13,13,13,13,13,13,13,5,18,18,13,1,8,7,13,12,3]

col_names = [
"tipo_registro",
"data_pregao",
"cod_bdi",
"cod_negociacao",
"tipo_mercado",
"nome_empresa",
"especificacao_papel",
"prazo_dias_merc_termo",
"moeda_referencia",
"preco_abertura",
"preco_maximo",
"preco_minimo",
"preco_medio",
"preco_ultimo_negocio",
"preco_melhor_oferta_compra",
"preco_melhor_oferta_venda",
"numero_negocios",
"quantidade_papeis_negociados",
"volume_total_negociado",
"preco_exercicio",
"ìndicador_correcao_precos",
"data_vencimento" ,
"fator_cotacao",
"preco_exercicio_pontos",
"codigo_isin",
"num_distribuicao_papel"]

decimal_config=[
"preco_abertura",
"preco_maximo",
"preco_minimo",
"preco_medio",
"preco_ultimo_negocio",
"preco_melhor_oferta_compra",
"preco_melhor_oferta_venda",
"volume_total_negociado",
"preco_exercicio",
"preco_exercicio_pontos"
]


remains = [
"data_pregao",
"cod_negociacao",
"tipo_mercado",
"nome_empresa",
"preco_abertura",
"preco_maximo",
"preco_minimo",
"preco_medio",
"preco_ultimo_negocio",
]

In [ ]:
dataset = pd.DataFrame()

In [ ]:
'''
year = 2022
while year > 1985:
    df = pd.read_fwf(f'{path}COTAHIST_A{year}.TXT',
                                    encoding='ISO-8859-1',
                                    header=0,
                                    widths=widths,
                                    skipfooter=1,
                                    engine='python',
                                    parse_dates=[1],
                                    infer_datetime_format=True,
                                    index_col=False)
    year -= 1

    #Definindo nomes das colunas
    df.columns = col_names

    #Corrigindo casas decimais
    for col in decimal_config:
        df[col]=df[col]/100

    #Selecionando colunas
    df = df[remains]
    
    #Mascara de empresas desejadas
    mask_empresas = df.cod_negociacao.isin(codigo)

    df = df[mask_empresas]

    dataset = pd.concat([dataset, df], ignore_index=True)
'''

In [ ]:
dataset

## Export DF

In [ ]:
path = r'C:\Users\Pedro\OneDrive\Desktop\Ironhack\05. Dados\B3\txt/'

dataset.to_csv(f'{path}dataset-IBRA.csv',
                    encoding='ISO-8859-1',
                    sep=';',
                    decimal='.',
                    index=False)

In [ ]:
pd.read_csv(f'{path}dataset-IBRA.csv',
                encoding='ISO-8859-1',
                sep=';',
                decimal='.',
                index_col=False
                )

## Yahoo finance

In [32]:
symbol_list_br = ['^BVSP','^IBX50']
symbol_list_eua = ['^DJI','^IXIC','^GSPC','GC=F']
symbol_list_crypto = ['BTC-USD', 'ETH-USD']

col_names = ['date',
"open",
"high",
"low",
"close",
'cod_yfinance']

drop_col = ['Volume','Dividends','Stock Splits']

start = '2000-01-01'
start_br ='2004-11-03'

In [33]:
df_br = pd.DataFrame()
df_eua = pd.DataFrame()
df_crypto = pd.DataFrame()

In [34]:
for ativo in symbol_list_br:
        chamada_api = yf.Ticker(ativo).history(start=start_br)
        chamada_api['cod_yfinace'] = ativo
        df_br = pd.concat([df_br, chamada_api])

In [35]:
df_br = df_br.drop(drop_col, axis = 1)
df_br.reset_index(inplace=True)
df_br['Date'] = df_br['Date'].dt.date
df_br.columns = col_names
df_br

,date,open,high,low,close,cod_yfinance
0,2004-11-03,23275.000000,23770.000000,23275.000000,23660.000000,^BVSP
1,2004-11-04,23660.000000,23963.000000,23629.000000,23880.000000,^BVSP
2,2004-11-05,23864.000000,24000.000000,23499.000000,23542.000000,^BVSP
3,2004-11-08,23542.000000,23542.000000,23072.000000,23212.000000,^BVSP
4,2004-11-09,23213.000000,23305.000000,23022.000000,23216.000000,^BVSP
...,...,...,...,...,...,...
6911,2022-11-11,18418.289062,18986.470703,18371.419922,18778.070312,^IBX50
6912,2022-11-14,18849.750000,19222.199219,18799.490234,19024.000000,^IBX50
6913,2022-11-16,19023.919922,19084.750000,18438.740234,18557.150391,^IBX50
6914,2022-11-17,18557.150391,18557.150391,18075.339844,18488.970703,^IBX50


In [40]:
path = r'C:\Users\Pedro\OneDrive\Desktop\Ironhack\04. GitHub\data_visualization\data/'

df_br.to_csv(f'{path}index_br.csv',
                    encoding='ISO-8859-1',
                    sep=';',
                    decimal='.',
                    index=False)

In [36]:
for ativo in symbol_list_eua:
        chamada_api = yf.Ticker(ativo).history(start=start)
        chamada_api['cod_yfinance'] = ativo
        df_eua = pd.concat([df_eua, chamada_api])

In [37]:
df_eua = df_eua.drop(drop_col, axis = 1)
df_eua.reset_index(inplace=True)
df_eua['Date'] = df_eua['Date'].dt.date
df_eua.columns = col_names
df_eua

,date,open,high,low,close,cod_yfinance
0,2000-01-03,11501.849609,11522.009766,11305.690430,11357.509766,^DJI
1,2000-01-04,11349.750000,11350.059570,10986.450195,10997.929688,^DJI
2,2000-01-05,10989.370117,11215.099609,10938.669922,11122.650391,^DJI
3,2000-01-06,11113.370117,11313.450195,11098.450195,11253.259766,^DJI
4,2000-01-07,11247.059570,11528.139648,11239.919922,11522.559570,^DJI
...,...,...,...,...,...,...
22850,2022-11-15,1770.800049,1773.800049,1769.900024,1773.800049,GC=F
22851,2022-11-16,1776.599976,1778.900024,1773.000000,1773.000000,GC=F
22852,2022-11-17,1763.000000,1763.000000,1756.000000,1760.800049,GC=F
22853,2022-11-18,1763.300049,1763.300049,1747.599976,1751.900024,GC=F


In [41]:
path = r'C:\Users\Pedro\OneDrive\Desktop\Ironhack\04. GitHub\data_visualization\data/'

df_eua.to_csv(f'{path}index_eua.csv',
                    encoding='ISO-8859-1',
                    sep=';',
                    decimal='.',
                    index=False)

In [38]:
for ativo in symbol_list_crypto:
        chamada_api = yf.Ticker(ativo).history(start=start)
        chamada_api['cod_yfinace'] = ativo
        df_crypto = pd.concat([df_crypto, chamada_api])

In [39]:
df_crypto = df_crypto.drop(drop_col, axis = 1)
df_crypto.reset_index(inplace=True)
df_crypto['Date'] = df_crypto['Date'].dt.date
df_crypto.columns = col_names
df_crypto

,date,open,high,low,close,cod_yfinance
0,2014-09-17,465.864014,468.174011,452.421997,457.334015,BTC-USD
1,2014-09-18,456.859985,456.859985,413.104004,424.440002,BTC-USD
2,2014-09-19,424.102997,427.834991,384.532013,394.795990,BTC-USD
3,2014-09-20,394.673004,423.295990,389.882996,408.903992,BTC-USD
4,2014-09-21,408.084991,412.425995,393.181000,398.821014,BTC-USD
...,...,...,...,...,...,...
4820,2022-11-16,1251.808838,1264.486328,1192.987793,1215.602539,ETH-USD
4821,2022-11-17,1215.848633,1224.965698,1189.152710,1200.808594,ETH-USD
4822,2022-11-18,1200.802124,1226.737427,1200.802124,1212.300293,ETH-USD
4823,2022-11-19,1212.215454,1227.837280,1200.475586,1218.426758,ETH-USD


In [42]:
path = r'C:\Users\Pedro\OneDrive\Desktop\Ironhack\04. GitHub\data_visualization\data/'

df_crypto.to_csv(f'{path}crypto.csv',
                    encoding='ISO-8859-1',
                    sep=';',
                    decimal='.',
                    index=False)

## Get S&P500 companies list

In [ ]:
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'

data = pd.read_html(url)

In [ ]:
data[0].head()

In [ ]:
sp500 = data[0].iloc[:,[0,1,3,4]]

In [ ]:
sp500.columns = ['ticker', 'name', 'sector', 'sub_industry']

In [ ]:
sp500